<a href="https://colab.research.google.com/github/kanta-yamano/atc/blob/main/%E6%9C%80%E9%81%A9%E5%8C%96%E6%B3%95%E3%81%AE%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下では、複雑なスケジューリング最適化問題を例題として取り上げる。ランダム選択、アニーリング法と遺伝アルゴリズムの比較を行う。
はじめにフライトスケジュールを与える。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/My Drive/大学院分散システム/schedule.txt') as fp:
  lines = fp.read().strip().split('\n') 
flights = {}
for line in lines:
  origin,dest,depart,arrive,price=line.split(',')
  flights.setdefault((origin,dest),[])
  # Add details to the list of possible flights
  flights[(origin,dest)].append((depart,arrive,int(price)))

In [ ]:
import time
import random
import math

people = [('Seymour','BOS'),
          ('Franny','DAL'),
          ('Zooey','CAK'),
          ('Walt','MIA'),
          ('Buddy','ORD'),
          ('Les','OMA')]
# Laguardia
destination='LGA'

In [ ]:
def getminutes(t):
  x=time.strptime(t,'%H:%M')
  return x[3]*60+x[4]

def printschedule(r):
  for d in range(int(len(r)/2.0)): #Arranged for Python3.4
    name=people[d][0]
    origin=people[d][1]
    out=flights[(origin,destination)][int(r[d])]
    ret=flights[(destination,origin)][int(r[d+1])]
    print( '%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (name,origin,
                                                  out[0],out[1],out[2],
                                                  ret[0],ret[1],ret[2]))
def schedulecost(sol):
  totalprice=0
  latestarrival=0
  earliestdep=24*60

  for d in range(int(len(sol)/2.0)):# Arranged for python3.4 by A.Fujii
    # Get the inbound and outbound flights
    origin=people[d][1]
    outbound=flights[(origin,destination)][int(sol[d])]
    returnf=flights[(destination,origin)][int(sol[d+1])]
    
    # Total price is the price of all outbound and return flights
    totalprice+=outbound[2]
    totalprice+=returnf[2]
    
    # Track the latest arrival and earliest departure
    if latestarrival<getminutes(outbound[1]): latestarrival=getminutes(outbound[1])
    if earliestdep>getminutes(returnf[0]): earliestdep=getminutes(returnf[0])
  
  # Every person must wait at the airport until the latest person arrives.
  # They also must arrive at the same time and wait for their flights.
  totalwait=0  
  for d in range(int(len(sol)/2.0)): # Arranged for python3.4 by A.Fujii
    origin=people[d][1]
    outbound=flights[(origin,destination)][int(sol[d])]
    returnf=flights[(destination,origin)][int(sol[d+1])]
    totalwait+=latestarrival-getminutes(outbound[1])
    totalwait+=getminutes(returnf[0])-earliestdep  

  # Does this solution require an extra day of car rental? That'll be $50!
  if latestarrival>earliestdep: totalprice+=50
  
  return totalprice+totalwait


以上の準備のもとで、4種類の最適化方法の比較を行う。

# ランダムオプティマイズ

In [ ]:
def randomoptimize(domain,costf):
  best=999999999
  bestr=None
  for i in range(0,1000):
    # Create a random solution
    r=[float(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
    
    # Get the cost
    cost=costf(r)
    
    # Compare it to the best one so far
    if cost<best:
      best=cost
      bestr=r 
  return r

初期スケジュールの状態とそれぞれのフライトコストを示す。

In [ ]:
s = [ 4,4,4,2,2,6,6,5,5,6,6,0]
printschedule(s)

   Seymour       BOS 12:34-15:02 $109 12:08-14:05 $142
    Franny       DAL 12:19-15:25 $342 12:20-16:34 $500
     Zooey       CAK 12:08-14:59 $149  9:58-12:56 $249
      Walt       MIA  9:15-12:29 $225  9:25-12:46 $295
     Buddy       ORD  9:42-11:32 $169 15:04-17:23 $189
       Les       OMA 15:03-16:42 $135 15:07-17:21 $129


ランダム最適化の結果


In [ ]:
domain = [(0,8)]*(len(people)*2)
print( ' ===== random optimization ======')
s = randomoptimize(domain,schedulecost)
print(s)
printschedule(s)
print( ' schedulecost = %s' %schedulecost(s))

 ===== random optimization ======
[6.0, 1.0, 8.0, 7.0, 7.0, 0.0, 5.0, 3.0, 0.0, 5.0, 2.0, 3.0]
   Seymour       BOS 15:27-17:18 $151  8:23-10:28 $149
    Franny       DAL  7:53-11:37 $433 18:44-22:42 $351
     Zooey       CAK 18:35-20:28 $204 16:33-18:15 $253
      Walt       MIA 17:07-20:04 $291 16:50-19:26 $304
     Buddy       ORD 16:43-19:00 $246  6:03- 8:43 $219
       Les       OMA  6:11- 8:31 $249 14:05-15:47 $226
 schedulecost = 7336


5000を超える値となる。

# ヒルクライム法

In [ ]:
def hillclimb(domain,costf):
  # Create a random solution
  sol=[random.randint(domain[i][0],domain[i][1])
      for i in range(len(domain))]
  # Main loop
  while 1:
    # Create list of neighboring solutions
    neighbors=[]
    
    for j in range(len(domain)):
      # One away in each direction
      if sol[j]>domain[j][0]:
        neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])
      if sol[j]<domain[j][1]:
        neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:])

    # See what the best solution amongst the neighbors is
    current=costf(sol)
    best=current
    for j in range(len(neighbors)):
      cost=costf(neighbors[j])
      if cost<best:
        best=cost
        sol=neighbors[j]

    # If there's no improvement, then we've reached the top
    if best==current:
      break
  return sol

実行結果を示す。

In [ ]:
#5.5 Hill Climb Search
domain = [(0,8)]*(len(people)*2)
print( ' ===== hill climb optimization ======')
s = hillclimb(domain,schedulecost)
print( s)
printschedule(s)
print( ' schedulecost = %s' %schedulecost(s))

 ===== hill climb optimization ======
[8, 8, -3, 8, 6, 8, 6, 2, 0, 3, 7, 0]
   Seymour       BOS 18:34-19:36 $136 18:24-20:49 $124
    Franny       DAL 18:26-21:29 $464 17:14-20:59 $277
     Zooey       CAK 17:08-19:08 $262 18:17-21:04 $259
      Walt       MIA 18:23-21:35 $134 15:23-18:49 $150
     Buddy       ORD 15:58-18:40 $173 18:33-20:22 $143
       Les       OMA 18:12-20:17 $242 15:07-17:21 $129
 schedulecost = 3804


#アニーリング法
同様にアニーリング（焼きなまし法）を定義する。

In [ ]:
def annealingoptimize(domain,costf,T=10000.0,cool=0.95,step=1):
  # Initialize the values randomly
  vec=[float(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
  
  while T>0.1:
    # Choose one of the indices
    i=random.randint(0,len(domain)-1)

    # Choose a direction to change it
    dir=random.randint(-step,step)

    # Create a new list with one of the values changed
    vecb=vec[:]
    vecb[i]+=dir
    if vecb[i]<domain[i][0]: vecb[i]=domain[i][0]
    elif vecb[i]>domain[i][1]: vecb[i]=domain[i][1]

    # Calculate the current cost and the new cost
    ea=costf(vec)
    eb=costf(vecb)
    p=pow(math.e,(-eb-ea)/T)

    # Is it better, or does it make the probability
    # cutoff?
    if (eb<ea or random.random()<p):
      vec=vecb      

    # Decrease the temperature
    T=T*cool
  return vec

In [ ]:
#5.6 Simulated Annealing
domain = [(0,8)]*(len(people)*2)
print( ' ===== annealing optimization ======')
s = annealingoptimize(domain,schedulecost)
print( s)
printschedule(s)
print( ' schedulecost = %s' %schedulecost(s))

NameError: ignored

# 遺伝アルゴリズム
遺伝アルゴリズムを定義し確認する。


In [ ]:
def geneticoptimize(domain,costf,popsize=50,step=1,
                    mutprob=0.2,elite=0.2,maxiter=100):
   # Mutation Operation
  def mutate_error (vec):
    i=random.randint(0,len(domain)-1)
    if random.random()<0.5 and vec[i]>domain[i][0]:
      return vec[0:i]+[vec[i]-step]+vec[i+1:] 
    elif vec[i]<domain[i][1]:
      return vec[0:i]+[vec[i]+step]+vec[i+1:]
    
  def mutate(vec):
    i=random.randint(0,len(domain)-1)
    # Corrected so different step values can be added
    if random.random()<0.5 and vec[i]-step>domain[i][0]:
      return vec[0:i]+[vec[i]-step]+vec[i+1:]
    elif vec[i]+step<domain[i][1]:
      return vec[0:i]+[vec[i]+step]+vec[i+1:]
    else:
      return vec
          
  # Crossover Operation
  def crossover(r1,r2):
    i=random.randint(1,len(domain)-2)
    return r1[0:i]+r2[i:]

  # Build the initial population
  pop=[]
  for i in range(popsize):
    vec=[random.randint(domain[i][0],domain[i][1]) 
         for i in range(len(domain))]
    pop.append(vec)
  
  # How many winners from each generation?
  topelite=int(elite*popsize)
  
  # Main loop
  for i in range(maxiter):
    scores=[(costf(v),v) for v in pop]
    scores.sort()
    ranked=[v for (s,v) in scores]
    # Start with the pure winers
    pop=ranked[0:topelite]

    # Add mutated and bred forms of the winners
    while len(pop)<popsize:
      if random.random()<mutprob:
        # Mutation
        c=random.randint(0,topelite)
        pop.append(mutate(ranked[c]))
      else:
        # Crossover
        c1=random.randint(0,topelite)
        c2=random.randint(0,topelite)
        pop.append(crossover(ranked[c1],ranked[c2]))
      # Print current best score
    #print( scores[0][0])
  return scores[0][1]

In [ ]:
#5.7 Genetic Algorithm
print( ' ===== genetic optimization ======')
domain = [(0,8)]*(len(people)*2)
s = geneticoptimize(domain,schedulecost)
print( s)
printschedule(s)
print( ' schedulecost = %s' %schedulecost(s))


 ===== genetic optimization ======
[4, 3, 3, 3, 4, 3, 3, 1, 1, 1, 1, 1]
   Seymour       BOS 12:34-15:02 $109 10:33-12:03 $ 74
    Franny       DAL 10:30-14:57 $290 10:51-14:16 $256
     Zooey       CAK 10:53-13:36 $189 10:32-13:16 $139
      Walt       MIA 11:28-14:40 $248 12:37-15:05 $170
     Buddy       ORD 12:44-14:17 $134 10:33-13:11 $132
       Les       OMA 11:08-13:07 $175 11:07-13:24 $171
 schedulecost = 2591


最も優れた解が得られることが分かる。